In [2]:
#!pip install gurobipy

In [2]:
##Importando las librerías necesarias
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
from recursos.clases import *
from recursos.heuristicas import *

In [4]:
#Definiendo los vehiculos y sus párametros asociados
veh1 = Vehiculo(1,180,carga=180)
veh2 = Vehiculo(2,90,carga=90)
veh3 = Vehiculo(3,80,carga=80)
flota = [veh1,veh2,veh3]

#Definiendo los destinos y sus párametros asociados
dest1 = Destino(1,70,3,90)
dest2 = Destino(2,90,3,0)
dest3 = Destino(3,190,3,1000)
dest4 = Destino(4,250,3,180)
destinos = [dest1,dest2,dest3,dest4]

#Definiendo las distancias
distancias = {
    (0,1):15,
    (0,2):10,
    (0,3):9,
    (0,4):24,
    (1,2):6,
    (1,3):5,
    (1,4):20,
    (2,3):1,
    (2,4):16,
    (3,4):21
}

In [6]:
numero_etapa = 1
condicion = "Continue"
list_resultados=[]
#Se crea el archivo donde se guarda el plan de distribución productos de la aplicación del modelo
with open('results/resultado.csv','w') as file:
    file.write("Etapa,")
    file.write("Vehiculo_id,")
    file.write("Destino_id,")
    file.write("Carga_entregada")
#Se crea un bucle que resuelve el modelo por etapas hasta que se establece la condición de alcanzar 
#el tiempo de cobertura planificado para cada destino
while condicion == "Continue":
    #Se genera un subconjunto de destinos que no alcanzan el tiempo de cobertura planificado, 
    #si el conjunto es vacío devuelve la condición ¨Stop¨ para salir del bucle
    sub_destinosXcobertura,condicion = reduccion_destinosXcobertura(list_destinos=destinos)
    if condicion=="Stop":
        break
    #Se genera un subconjunto de destinos en correspondencia con el numero de vehículos 
    #(en cada etapa un vehículo solo puede visitar un destino)
    sub_destinosXvehiculos = reduccion_destinosXvehiculos(list_destinos=sub_destinosXcobertura,list_vehiculos=flota)
    #Se establecen los parámetros de carga de mercancías e indicador tráfico generado 
    #en dependencia de las combinaciones entre vehículos y destinos
    parametros_carga_vehXdestino = determinar_cant_mercancias_vehiculosXdestino(list_destinos=sub_destinosXvehiculos,list_vehiculos=flota)
    parametros_indicador_trafico = determinar_indicador_trafico(list_destinos=sub_destinosXvehiculos,list_vehiculos=flota,params_carga_transportada=parametros_carga_vehXdestino,matrix_distancias=distancias)
    #Se establece la etapa y se inicializa y resuelve el modelo matematico correspondiente a la misma
    #se guardan los resultados
    nombre_etapa = f"Etapa {numero_etapa}"
    modelo_etapa = ModeloOpt(name_model=nombre_etapa,list_vehiculos=flota,list_destinos=sub_destinosXvehiculos,params_FO=parametros_indicador_trafico,params_carga=parametros_carga_vehXdestino)
    resultado = modelo_etapa.solve()
    list_resultados.append(resultado)
    #Se actualizan los parámetros de vehículos y destinos de acuerdo al resultado de la etapa
    actualizar_destinos(destinos=sub_destinosXvehiculos,carga_veh_destino=parametros_carga_vehXdestino,resultado=resultado)
    actualizar_vehiculos(vehiculos=flota,carga_veh_destino=parametros_carga_vehXdestino,resultado=resultado)
    numero_etapa+=1

resultados = pd.read_csv('results/resultado.csv',index_col='Etapa')
display(Markdown("### Resultados de la distribución"))
display(resultados)

with open('results/resultados_ObjVal.txt','w') as file:
    file.write(str(list_resultados))

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-3427U CPU @ 1.80GHz, instruction set [SSE2|AVX]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xe975820a
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Found heuristic solution: objective 27.3333333

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

### Resultados de la distribución

,Vehiculo_id,Destino_id,Carga_entregada
Etapa,,,
Etapa 1,1,2,180
Etapa 1,2,1,90
Etapa 1,3,4,80
Etapa 2,1,4,180
Etapa 2,2,2,90
Etapa 2,3,1,30
Etapa 3,1,4,180
Etapa 4,1,4,130
